
# Quickstart 3 - Investments & Storage


## Task Description

Consider a

- location is Seville, Spain,
- constant demand centre with a load of 100 MW,
- grid electricity at a price of 120 €/MWh (no feed-in to the grid allowed),
- can build a solar PV plant with annuitised costs of 50 €/kW/a and [this](https://model.energy/data/time-series-f17c3736a2719ce7da58484180d89e2d.csv) capacity factor time series
- can build a battery storage system with annuitised costs of 20 €/kW/a and 1 €/kWh/a, a round-trip efficiency of 90%, and a energy-to-power-ratio of 4 hours.

## PyPSA Solution



In [ ]:
import pypsa
import pandas as pd
import numpy as np

n = pypsa.Network()

n.add("Bus", "seville")

n.add("Load", "demand", bus="seville", p_set=100)

n.add("Generator", "grid", bus="seville", p_nom=100, marginal_cost=120, carrier="grid")



INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 130.70it/s]
INFO:linopy.io: Writing time: 0.36s


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 43803 primals, 105122 duals
Objective: 4.80e+07
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


('ok', 'optimal')

In [ ]:



p_max_pu = pd.read_csv(
    "https://model.energy/data/time-series-f17c3736a2719ce7da58484180d89e2d.csv",
    index_col=0,
    parse_dates=True,
)["solar"]
p_max_pu.head(20)


In [ ]:

n.set_snapshots(p_max_pu.index)


In [ ]:

n.add(
    "Generator",
    "solar",
    bus="seville",
    p_max_pu=p_max_pu,
    capital_cost=50_000,
    p_nom_extendable=True,
    carrier="solar",
)


In [ ]:

n.add(
    "StorageUnit",
    "battery",
    bus="seville",
    capital_cost=20_000 + 4 * 1_000,
    p_nom_extendable=True,
    carrier="battery",
    efficiency_store=np.sqrt(0.9),
    efficiency_dispatch=np.sqrt(0.9),
    max_hours=4,
)


In [ ]:

n.optimize(log_to_console=False)


In [ ]:
totex = {"opex": n.statistics.opex(), "capex": n.statistics.capex()}
pd.concat(totex, axis=1).div(1e6).round(2) # M€/a

opex  capex
component   carrier              
Generator   grid     12.45    NaN
            solar      NaN  29.17
StorageUnit battery    NaN   8.79

In [52]:
(n.statistics.capex().sum() + n.statistics.opex().sum()) / 100 / 8760 # €/MWh

np.float64(57.54934839091324)

In [ ]:
n.statistics.energy_balance().div(1e3) # GWh

component    carrier  bus_carrier
Generator    grid     AC             103.711759
             solar    AC             817.682557
Load         -        AC            -876.000000
StorageUnit  battery  AC             -45.394316
dtype: float64

In [53]:
n.storage_units_t.state_of_charge.loc["2011-01"].plot(backend="plotly")

In [ ]:
n.add(
    "Carrier",
    ["grid", "solar", "battery", "AC"],
    color=["blue", "yellow", "green", "k"],
)

n.statistics.energy_balance.iplot()


Find many more extensive examples in the [examples](examples.md) section.

The [user guide](user-guide.md) section contains detailed information on architecture, components, problem formulation and utilities.